# A Custom Multi Layer Perceptron For Time Series Classification

### Importing Three Different Time Series Datasets by using sktime's `load_UCR_UEA_dataset` 
#### About the datasets:
* First dataset is called **DistalPhalanxOutlineCorrect**. It has 2 classes, It is of type Image, and it has one dimension.
* Second dataset is called **ItalyPowerDemand**. It has 2 classes, It is of type Sensor, and it has one dimension.
* Third dataset is called **BME**. It has 3 classes, It is of type Simulated, and it has one dimension.

In [ ]:
# ------ Importing necessary packages!
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# load_UCR_UEA_dataset
from sktime.datasets import load_UCR_UEA_dataset

In [2]:
# ------ Importing Time Series Datasets!
# 2 Classes, 1D, Image
X_train_1, y_train_1 = load_UCR_UEA_dataset(name="DistalPhalanxOutlineCorrect", split="train", return_type="numpy2D")
X_test_1, y_test_1 = load_UCR_UEA_dataset(name="DistalPhalanxOutlineCorrect", split="test", return_type="numpy2D")

# 2 Classes, 1D, Sensor 
X_train_2, y_train_2 = load_UCR_UEA_dataset(name= "ItalyPowerDemand", split="train", return_type="numpy2D")
X_test_2, y_test_2 = load_UCR_UEA_dataset(name="ItalyPowerDemand", split="test", return_type="numpy2D")

# 3Classes, 1D, Simulated
X_train_3, y_train_3 = load_UCR_UEA_dataset(name = "BME", split="train", return_type="numpy2D")
X_test_3, y_test_3 = load_UCR_UEA_dataset(name = "BME", split="test", return_type="numpy2D")

In [3]:
print("Dataset 1:", X_train_1.shape, X_test_1.shape)
print("Dataset 2:", X_train_2.shape, X_test_2.shape)
print("Dataset 3:", X_train_3.shape, X_test_3.shape)

Dataset 1: (600, 80) (276, 80)
Dataset 2: (67, 24) (1029, 24)
Dataset 3: (30, 128) (150, 128)


We have different datasets, which means we must have a function that can take in different datasets with different length.  

The plain baseline is a basic **Multi Layer Perceptron** which is build by stacking three fully connected layers. 

The fully connected layers **each has 500 neurons** following 2 designs roles:
* Using **Dropout** at each layer's input to improve the generalization capability!  
    * The fraction of the input units to drop on the input layer is **0.1**
    * The fraction of the input units to drop on the second and third layers is **0.2**
    * The fraction of the input units to drop on the output layer is **0.3**
* Using **ReLU** as activation function to prevent saturation of the gradient when the network is deep!  
And the output layer ends the network with a **softmax** activation function!

The MLP Model is then trained with **Adadelta** with learning rate 0.1, ρ = 0.95 and e = 1e − 8!

In [4]:
def create_mlp_model(input_shape, num_classes):
    mlp_model = tf.keras.Sequential([
        keras.layers.Dropout(0.1, input_shape= input_shape),
        keras.layers.Dense(500, activation="relu"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(500, activation="relu"),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(500, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(num_classes, activation="softmax")
    ])
    mlp_model.compile(optimizer=tf.keras.optimizers.Adadelta(learning_rate=0.1, rho=0.95, epsilon=1e-8),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return mlp_model

Defining **callbacks** parameters for fine tuning!

In [5]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
]

Creating training sets by using **train_test_split()** to split the data into random train and test subsets!  
* test_size = 0.2! 
    * 20% of the data will be used for the valiation/test set and the remaining 80% will be used for the training set!
* random_state=42!
    * Ensuring that the splits that generates are reproducible!

In [6]:
# --------- Creating and training MLP on the FIRST dataset ----------- 
X_train_MLP_1, X_val_MLP_1, y_train_MLP_1, y_val_MLP_1 = train_test_split(X_train_1, y_train_1, test_size=0.2, random_state=42)
# --------- Creating and training MLP on the SECOND dataset ----------- 
X_train_MLP_2, X_val_MLP_2, y_train_MLP_2, y_val_MLP_2 = train_test_split(X_train_2, y_train_2, test_size=0.2, random_state=42)
# ------------- Creating and training MLP on the Third dataset ------------
X_train_MLP_3, X_val_MLP_3, y_train_MLP_3, y_val_MLP_3 = train_test_split(X_train_3, y_train_3, test_size=0.2, random_state=42)